<p>Измалкова Дарья БКЛ182</p>

<h1>Домашнее задание №2 часть 1</h1>

In [1]:
from pymystem3 import Mystem
import nltk
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from pymorphy2 import MorphAnalyzer
from collections import Counter
import json
import re
import os


morph = MorphAnalyzer()
m = Mystem()

<h3>Для обработки взята книга Ф.М. Достоевского "Игрок"</h3>

In [2]:
with open('igrok.txt', encoding='utf-8') as f:
    text = f.read()

<h3>Парсинг с помощью Mystem</h3>
<p>Питоновский модуль работал слишком медленно (больше нескольких часов), поэтому я воспользовалась консольной версией</p>
<p>Опции "-gin" - печатать грамматическую информацию, склеивать информацию словоформ при одной лемме, каждое слово с новой строки</p>

In [3]:
%%time
import os

inp = "igrok.txt"
outp = "igrok.json"
mystem_path = os.path.join('/Users/User', 'mystem.exe')

input_filename = os.path.abspath(inp)
output_filename = os.path.abspath(outp)
os.system(f"{mystem_path} {'-gin'} {'--format json'} {input_filename} {output_filename}") 

Wall time: 947 ms


0

<p>Проверка, что файл нормально обработался и записался</p>

In [4]:
ana_list = []
with open('igrok.json', encoding='utf-8') as f:
    text = f.read()
    lines = text.splitlines()

for line in lines:
    data = json.loads(line)
    ana_list.append(data)

print(ana_list[100:105])

[{'analysis': [{'lex': 'сестра', 'gr': 'S,жен,од=(пр,ед|дат,ед)'}], 'text': 'сестре'}, {'analysis': [{'lex': 'быть', 'gr': 'V,нп=прош,ед,изъяв,сред,несов'}, {'lex': 'было', 'gr': 'PART='}], 'text': 'Было'}, {'analysis': [{'lex': 'ясно', 'gr': 'ADV='}, {'lex': 'ясный', 'gr': 'A=ед,кр,сред'}], 'text': 'ясно'}, {'analysis': [{'lex': 'что', 'gr': 'CONJ='}, {'lex': 'что', 'gr': 'SPRO,ед,сред,неод=(вин|им)'}, {'lex': 'что', 'gr': 'ADVPRO='}], 'text': 'что'}, {'analysis': [{'lex': 'они', 'gr': 'SPRO,мн=им'}], 'text': 'они'}]


<h3>Токенизация с помощью nltk</h3>

In [5]:
(word_tokenize(text))
# убрала пунктуацию и привела к нижнему регистру
words = [w.lower() for w in word_tokenize(text) if w.isalpha()]

<h3>Парсинг с помощью pymorphy</h3>
<p>Чтобы было удобнее потом сохранить результат в json, сразу перевожу данные в список из словарей</p>

In [6]:
%%time
analist = []

for word in words:
    ana = morph.parse(word)
    first = ana[0] # использую только 1-й вариант разбора

    word = first.word # слово
    tag = first.tag # грам. разбор
    lemma = first.normal_form # лемма
    prob = first.score # вероятность

    ana_dict = {'text' : word, 'tag' : [tag], 'lemma' : lemma, 'prob' : prob}
    analist.append(ana_dict)

Wall time: 46.7 s


<p>Чтобы можно было вытаскивать теги, не используя встроенные функции OpencorporaTag, добавила их отдельно в словарь, в список в значении "tag".</p>

In [7]:
for parse in analist:
    new_thing = {}
    if parse['tag'][0].POS:
        new_thing.update({'POS':parse['tag'][0].POS})
    if parse['tag'][0].animacy:
        new_thing.update({'animacy':parse['tag'][0].animacy})
    if parse['tag'][0].aspect:
        new_thing.update({'aspect':parse['tag'][0].aspect})
    if parse['tag'][0].case:
        new_thing.update({'case':parse['tag'][0].case})
    if parse['tag'][0].gender:
        new_thing.update({'gender':parse['tag'][0].gender})
    if parse['tag'][0].involvement:
        new_thing.update({'involvement':parse['tag'][0].involvement})
    if parse['tag'][0].mood:
        new_thing.update({'mood':parse['tag'][0].mood})
    if parse['tag'][0].number:
        new_thing.update({'number':parse['tag'][0].number})
    if parse['tag'][0].person:
        new_thing.update({'person':parse['tag'][0].person})
    if parse['tag'][0].tense:
        new_thing.update({'tense':parse['tag'][0].tense})
    if parse['tag'][0].transitivity:
        new_thing.update({'transitivity':parse['tag'][0].transitivity})
    if parse['tag'][0].voice:
        new_thing.update({'voice':parse['tag'][0].voice})
    parse['tag'].append(new_thing)

print(analist[100:105])

[{'text': 'достоевский', 'tag': [OpencorporaTag('NOUN,anim,masc,Sgtm,Surn sing,nomn'), {'POS': 'NOUN', 'animacy': 'anim', 'case': 'nomn', 'gender': 'masc', 'number': 'sing'}], 'lemma': 'достоевский', 'prob': 1.0}, {'text': 'analysis', 'tag': [OpencorporaTag('LATN'), {}], 'lemma': 'analysis', 'prob': 1.0}, {'text': 'lex', 'tag': [OpencorporaTag('LATN'), {}], 'lemma': 'lex', 'prob': 1.0}, {'text': 'фёдор', 'tag': [OpencorporaTag('NOUN,anim,masc,Name sing,nomn'), {'POS': 'NOUN', 'animacy': 'anim', 'case': 'nomn', 'gender': 'masc', 'number': 'sing'}], 'lemma': 'фёдор', 'prob': 0.8}, {'text': 'gr', 'tag': [OpencorporaTag('LATN'), {}], 'lemma': 'gr', 'prob': 1.0}]


<p>Пришлось оставить исходные значения "tag", так как не все грамматические значения относятся к какому-либо аттрибуту.</p>
<p>Как результат, в значении "tag" список из двух елементов, 1-й - теги в формате OpencorporaTag, 2-й - словарь вида "аттрибут : тег".</p>

<h3>Сохранение в формате json</h3>
<p>Формат json не признает OpencorporaTag, поэтому переделала тег в строку и убрала "OpencorporaTag", оставив только непосредственно теги</p>

In [8]:
with open('igrok_parse.json', 'w', encoding='utf-8') as f:

    for parse in analist:
        parse['tag'][0] = str(parse['tag'][0])
        parse['tag'][0] = re.sub(r'OpencorporaTag()', '', parse['tag'][0])

        json.dump(parse, f)
        f.write('\n')

<p>Проверка, что файл нормально записался, OpencorporaTag исчез.</p>

In [9]:
ana_list = []
with open('igrok_parse.json', encoding='utf-8') as f:
    text = f.read()
    lines = text.splitlines()

for line in lines:
    data = json.loads(line)
    ana_list.append(data)

print(ana_list[100:105])

[{'text': 'достоевский', 'tag': ['NOUN,anim,masc,Sgtm,Surn sing,nomn', {'POS': 'NOUN', 'animacy': 'anim', 'case': 'nomn', 'gender': 'masc', 'number': 'sing'}], 'lemma': 'достоевский', 'prob': 1.0}, {'text': 'analysis', 'tag': ['LATN', {}], 'lemma': 'analysis', 'prob': 1.0}, {'text': 'lex', 'tag': ['LATN', {}], 'lemma': 'lex', 'prob': 1.0}, {'text': 'фёдор', 'tag': ['NOUN,anim,masc,Name sing,nomn', {'POS': 'NOUN', 'animacy': 'anim', 'case': 'nomn', 'gender': 'masc', 'number': 'sing'}], 'lemma': 'фёдор', 'prob': 0.8}, {'text': 'gr', 'tag': ['LATN', {}], 'lemma': 'gr', 'prob': 1.0}]


<h3>Доля частей речи</h3>

In [10]:
word_count = len(analist) # количество слов в тексте
poslist = [] # список частей речи

# у некоторых слов отсутсвует часть речи (иноязычные слова, римские цифры)
for parse in analist:
    if 'POS' in parse['tag'][1].keys():
        poslist.append(parse['tag'][1]['POS'])

# считает количество вхождений
posdict = Counter(poslist)
# находит и выводит долю
for pos, count in posdict.items():
    print('Доля', pos, str(round(count/word_count * 100, 2))+'%')

Доля NOUN 14.57%
Доля PREP 2.22%
Доля ADJF 2.59%
Доля ADVB 1.56%
Доля NPRO 2.29%
Доля INFN 2.0%
Доля GRND 0.88%
Доля VERB 1.23%
Доля NUMR 0.18%
Доля CONJ 4.19%
Доля PRCL 1.68%
Доля INTJ 0.06%
Доля PRED 0.13%
Доля COMP 0.03%
Доля ADJS 0.13%
Доля PRTS 0.02%
Доля PRTF 0.06%


<h3>Топ-20 глаголов и наречий</h3>

In [11]:
def top_pos(analist,tag):
    lemmalist = [] # список лемм части речи
    top_list = [] # список топ-20

# находит лемму если в разборе соответсвующая часть речи   
    for parse in analist:
        if tag in parse['tag'][0]:
            lemmalist.append(parse['lemma'])
# cчитает вхождения
    lemmadict = Counter(lemmalist)
# находит 20 самых частотных
    for key, value in lemmadict.most_common(20):
        top_list.append(key)

    return top_list
    
    
print('Топ-20 глаголов:',top_pos(analist,'VERB'))
print('Топ-20 наречий:',top_pos(analist,'ADVB'))

Топ-20 глаголов: ['быть', 'мочь', 'крупереть', 'знать', 'сказать', 'хотеть', 'начать', 'говорить', 'стать', 'думать', 'видеть', 'выйти', 'проиграть', 'заметить', 'дать', 'взять', 'понимать', 'любить', 'смотреть', 'отвечать']
Топ-20 наречий: ['уж', 'тоже', 'уже', 'теперь', 'много', 'тут', 'вдруг', 'очень', 'почти', 'опять', 'здесь', 'потому', 'точно', 'тогда', 'действительно', 'тотчас', 'наконец', 'довольно', 'прямо', 'узко']


<h3>N-граммы</h3>

In [12]:
def find_ngrams(text, number):
# делает список всех н-грамм
    ngram_list = list(ngrams(text, number))
# создает частотный словарь
    ngrams_rank = Counter(ngram_list)
# находит 25 самых частотных
    top_20 = ngrams_rank.most_common(25)
# убирает все лишние скобки
    for ngram in top_20:
        n_words = ' '.join(ngram[0])
        frequency = ngram[1]
        
        print(n_words, frequency)

<p><b>Биграммы</b></p>

In [13]:
find_ngrams(words, 2)

analysis lex 46207
gr s 20511
gr lex 19686
ед text 12386
gr text 12202
ед ед 10183
gr v 9281
gr spro 8829
пр ед 8770
lex и 8356
и gr 8356
s пр 6139
муж ед 5540
вин ед 5277
ед lex 4972
ед сред 4649
s муж 4623
сред text 3890
мн мн 3871
ед изъяв 3835
мн text 3642
v несов 3598
ед полн 3565
s жен 3504
lex я 3492


<p><b>Триграммы</b></p>

In [14]:
find_ngrams(words, 3)

lex и gr 8356
ед ед ед 6291
gr s пр 6139
s пр ед 6138
пр ед text 4900
gr s муж 4623
и gr lex 4178
gr lex и 4178
gr v несов 3598
gr s жен 3504
lex я gr 3492
gr spro lex 3228
мн мн мн 3120
lex что gr 2913
gr spro ед 2883
spro ед сред 2661
ед сред lex 2655
gr spro text 2610
пр ед lex 2581
муж ед text 2549
gr v ед 2526
gr вин ед 2334
s муж ед 2210
ед изъяв муж 2179
analysis lex я 2164
